# Практическое занятие №5

*П.Н. Советов, РТУ МИРЭА*

Многие задачи из этого блокнота решать удобнее в парном составе: один студент занимается написанием кода программы и пытается внести в код такие ошибки, которые будет сложно выявить с помощью тестов другому студенту.

Далее потребуется несколько программ, содержащих ошибки. Если в задаче явно не указана программа для тестирования, то используется одна из следующих программ:


1. Сортировка.

In [ ]:
def bucketsort(arr, k):
    counts = [0] * k
    for x in arr:
        counts[x] += 1

    sorted_arr = []
    for i in range(k):
        sorted_arr.extend([i] * counts[i])

    return sorted_arr


2. Бинарный поиск.

In [ ]:
def binary_search(arr, x):
    left = 0
    right = len(arr)
    while left <= right:
        mid = round((left + right) / 2)
        if arr[mid] == x:
            return mid
        if arr[mid] < x:
            left = mid + 1
        else:
            right = mid
    return -1


3. Вычисление расстояния между точками.

In [ ]:
def distance(x1, y1, x2, y2):
    return ((x2 + x1)**2 - (y2 + y1)**2) ** 0.25

4. Определение типа треугольника.

In [ ]:
def triangle_type(x1, y1, x2, y2, x3, y3):
    a = distance(x1, y1, x2, y2)
    b = distance(x2, y2, x3, y3)
    c = distance(x3, y3, x1, y1)
    if a == b == c:
        return "равнобедренный"
    elif a == b or a == c or b == c:
        return "равносторонний"
    elif a != b != c:
        return "разносторонний"


5. Сжатие и расжатие данных по методу RLE.

In [ ]:
def encode_rle(data):
    encoded = bytes()
    count = 0
    last_char = data[-1]
    for i in range(1, len(data) + 1):
        if data[i] == last_char:
            count += 1
        else:
            encoded.append(data[i])
            encoded.append(count)
            count = 1
            last_char = data[i]
    encoded.append(count)
    encoded.append(last_char)
    return bytes(encoded)

def decode_rle(data):
    decoded = bytes()
    i = 1
    while i < len(data):
        count = data[i - 1]
        char = data[i]
        decoded.extend([char]*count)
        i += 1
    return bytes(decoded)


6. Банковский счет.

In [ ]:
class BankAccount:
    def __init__(self, account_number, balance=0):
        self.account_number = account_number
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        return f"{amount} средств успешно зачислены на счет {self.account_number}"

    def withdraw(self, amount):
        self.balance -= amount
        return f"{amount} средств успешно сняты с счета {self.account_number}"

    def check_balance(self):
        return f"Баланс счета {self.account_number}: {self.balance}"

7. Банковский счет с использованием базы данных.

In [ ]:
import sqlite3

class BankAccount:
    def __init__(self, account_number):
        self.account_number = account_number
        self.conn = sqlite3.connect('bank.db')
        self.cursor = self.conn.cursor()
        self.cursor.execute(
            "CREATE TABLE IF NOT EXISTS accounts (account_number INTEGER PRIMARY KEY, balance REAL)")
        self.conn.commit()

    def deposit(self, amount):
        self.cursor.execute(
            "UPDATE accounts SET balance = balance + ? WHERE account_number = ?", (amount, self.account_number))
        self.conn.commit()
        return f"{amount} средств успешно зачислены на счет {self.account_number}"

    def withdraw(self, amount):
        self.cursor.execute(
            "SELECT balance FROM accounts WHERE account_number = ?", (self.account_number,))
        balance = self.cursor.fetchone()[0]
        self.cursor.execute(
            "UPDATE accounts SET balance = balance - ? WHERE account_number = ?", (amount, self.account_number))
        self.conn.commit()
        return f"{amount} средств успешно сняты с счета {self.account_number}"

    def check_balance(self):
        self.cursor.execute(
            "SELECT balance FROM accounts WHERE account_number = ?", (self.account_number,))
        balance = self.cursor.fetchone()[0]
        return f"Баланс счета {self.account_number}: {balance}"

    def close_account(self):
        self.cursor.execute(
            "DELETE FROM accounts WHERE account_number = ?", (self.account_number,))
        self.conn.commit()
        return f"Счет {self.account_number} закрыт"

    def create_account(self, balance):
        self.cursor.execute(
            "INSERT INTO accounts (account_number, balance) VALUES (?, ?)", (self.account_number, balance))
        self.conn.commit()
        return f"Счет {self.account_number} успешно создан"

## 1. Вводные задачи

**1.1.** (уровень сложности: низкий)

Исправьте функцию distance. Добавьте документацию к функции в виде docstring-строки. Укажите примеры в формате doctest. Примеры должны охватывать граничные случаи. Протестируйте программу с помощью вызова модуля doctest. Перенесите примеры в отдельный файл и снова протестируйте программу.

In [13]:
def distance(x1, y1, x2, y2):
    '''
    >>> distance(5, 2, 2, 6)
    5
    '''
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5


if __name__ == "__main__":
    import doctest
    doctest.testmod()
#distance(5, 2, 2, 6)

**********************************************************************
File "__main__", line 3, in __main__.distance
Failed example:
    distance(5, 2, 2, 6)
Expected:
    5
Got:
    5.0
**********************************************************************
1 items had failures:
   1 of   1 in __main__.distance
***Test Failed*** 1 failures.


**1.2.** (уровень сложности: высокий)

Добавьте к функции сортировки тестирование на случайных данных. Исправьте ошибки в функции.

Напишите к функции сортировки отдельную функцию-спецификацию в виде набора assert'ов.  Спецификация должна исчерпывающим образом описывать задачу сортировки (без привлечения готовых функций сортировки), иными словами – для **общего случая** нельзя придумать такое искажение кода сортировки, которое будет принято спецификацией.

In [21]:
import random
def bucketSort(array):
    largest = max(array)
    length = len(array)
    size = largest/length

    # Create Buckets
    buckets = [[] for i in range(length)]

    # Bucket Sorting
    for i in range(length):
        index = int(array[i]/size)
        if index != length:
            buckets[index].append(array[i])
        else:
            buckets[length - 1].append(array[i])

    # Sorting Individual Buckets
    for i in range(len(array)):
        buckets[i] = sorted(buckets[i])


    # Flattening the Array
    result = []
    for i in range(length):
        result = result + buckets[i]

    return result

In [24]:
def spetific():
    assert bucketSort([3,5,2,8])
    assert bucketSort([0,1,7,2,9])


def test_sort_random_data():

    data = [random.randint(1, 100) for _ in range(10)]

    #sorted_data = bucketsort(data)
    sorted_data = bucketSort(data)

    assert sorted_data == sorted(data)

test_sort_random_data()
spetific()

In [50]:
import random


def bucketsort(arr, k):
  buckets = [[] for _ in range(k)]

  for x in arr:
    if x >= k:
      pass
    else:
      buckets[x].append(x)

  sorted_arr = []
  for bucket in buckets:
    sorted_bucket = sorted(bucket)
    sorted_arr.extend(sorted_bucket)

  return sorted_arr


def spetific():
    assert bucketsort([3,5,2,8], 4)
    assert bucketsort([0,1,7,2,9], 5)


def test_sort_random_data():

    data = [random.randint(1, 100) for _ in range(10)]

    sorted_data = bucketsort(data, 100)

    assert sorted_data == sorted(data)

test_sort_random_data()
spetific()


**1.3.** (уровень сложности: средний)

Реализуйте конструкцию raises с помощью менеджера контекста в духе таковой из pytest.

Пример использования:

```Python
with raises(MealyError) as e:
    ...
```

In [17]:
import pytest

def test_zero_division():
    with pytest.raises(ZeroDivisionError):
        1 / 0


test_zero_division()

## 2. Библиотеки pytest и coverage

**2.1.** (уровень сложности: средний)

Научитесь работать с модулем pytest. Выберите одну из программ, содержащих ошибки. Создайте отдельный файл для тестирования, в который поместите тестирующие функции (не менее двух). Упростите код с помощью добавления fixture-функций. Добавьте параметризацию.

**2.2.** (уровень сложности: средний)

Выберите одну из программ, содержащих ошибки. Добавьте туда ввод со стороны пользователя. Добавьте макетный код для тестирования, с учетом такого ввода.

**2.2.** (уровень сложности: средний)

Научитесь работать с модулем coverage. Выберите одну из программ, содержащих ошибки. Получите статистику по покрытию операторов. Получите статистику по покрытию ветвей. Найдите случай, когда покрытие ветвей отличается от покрытия операторов. Постарайтесь изменить код исходной программы так, чтобы затруднить получение 100% покрытия. Найдите простой пример ошибки в выбранной программе, при полученном 100% покрытии. Реализуйте вывод статистики о покрытии в HTML-представлении с демонстрацией покрытия по строкам программы.

## 3. Мутационное тестирование

Прототип системы мутационного тестирования приведен ниже. Попробуйте разобраться в том, как работает этот код. Вам поможет документация к модулю ast.

Функция `mut_test` принимает на вход тестируемую функцию и функцию, осуществляющую тестирование с помощью `assert`.

In [ ]:
import random
from collections import defaultdict
import inspect
import ast


class Mutator(ast.NodeTransformer):
    def visit_Constant(self, node):
        # TODO
        return node


def mutate_code(src):
    tree = ast.parse(src)
    Mutator().visit(tree)
    return ast.unparse(tree)


def make_mutants(func, size):
    mutant = src = ast.unparse(ast.parse(inspect.getsource(func)))
    mutants = [src]
    while len(mutants) < size + 1:
        while mutant in mutants:
            mutant = mutate_code(src)
        mutants.append(mutant)
    return mutants[1:]


def mut_test(func, test, size=20):
    survived = []
    mutants = make_mutants(func, size)
    for mutant in mutants:
        try:
            exec(mutant, globals())
            test()
            survived.append(mutant)
        except:
            pass
    return survived


**3.1.** (уровень сложности: средний)

Выберите одну из программ, содержащих ошибки. Доработайте код мутационного тестирования так, чтобы генерировались программы-мутанты со случайными константами. Покажите, что при 100% покрытии тестами мутационное тестирование в состоянии находить ошибки.

In [48]:
import random
from collections import defaultdict
import inspect
import ast

class Mutator(ast.NodeTransformer):
    def visit_Constant(self, node):
        if isinstance(node.value, (int, float)):
            if abs(node.value) < 10:
                new_value = random.randint(-10, 10)
            elif abs(node.value) < 100:
                new_value = random.randint(-100, 100)
            else:
                new_value = random.uniform(-1000, 1000)

            node.value = new_value
        return node


In [49]:
def add_numbers(a, b):
  """
  Складывает два числа.

  Args:
      a: Первое число.
      b: Второе число.

  Returns:
      Сумму a и b.
  """
  return a + b

if __name__ == "__main__":
  result = add_numbers(10, 5)
  print(f"Сумма: {result}")


Сумма: 15


In [ ]:
def test_add_numbers():
  assert add_numbers(10, 5) == 15  # Correct test case
  # assert add_numbers(10, "5") == 15  # Uncomment to test error


In [ ]:
import mut_test

survived = mut_test(add_numbers, test_add_numbers, size=100)
print(f"Survived Mutants: {survived}")


**3.2.** (уровень сложности: средний)

Добавьте к системе мутационного тестирования генерацию случайных бинарных операций. Проверьте результат на сортировке. Постарайтесь генерировать программы-мутанты, которые «выживают» после большинства assert'ов.

In [ ]:
class Mutator(ast.NodeTransformer):

    def visit_BinOp(self, node):
        if isinstance(node.op, ast.Add) or isinstance(node.op, ast.Sub):
            # Мутируем операции сложения и вычитания
            new_op = random.choice([ast.Add, ast.Sub])  # Замена операции
            node.op = new_op
        elif isinstance(node.op, ast.Mult) or isinstance(node.op, ast.Div):
            # Мутируем операции умножения и деления
            new_op = random.choice([ast.Mult, ast.Div])  # Замена операции
            node.op = new_op
        return node


In [ ]:
def bubble_sort(data):
    """
    Сортирует список с помощью пузырьковой сортировки.

    Args:
        data (list): Список элементов.

    Returns:
        list: Отсортированный список.
    """
    for i in range(len(data) - 1):
        for j in range(len(data) - i - 1):
            if data[j] > data[j + 1]:
                data[j], data[j + 1] = data[j + 1], data[j]
    return data

def test_bubble_sort():
    assert bubble_sort([]) == []
    assert bubble_sort([5, 2, 4, 1, 3]) == [1, 2, 3, 4, 5]
    assert bubble_sort([-3, 0, 10, 5, -2]) == [-3, -2, 0, 5, 10]

survived_mutants = mut_test(bubble_sort, test_bubble_sort, size=100)

print(f"Выжившие мутанты: {survived_mutants}")


In [ ]:
class Mutator(ast.NodeTransformer):
  def visit_Constant(self, node):
    # ... (same as before)

  def visit_Binop(self, node):
    # Generate a random operator replacement
    if random.random() < 0.5:  # 50% chance of replacing the operator
      new_op = random.choice([ast.Add, ast.Sub, ast.Mult, ast.Div])
      if new_op != node.op:  # Avoid replacing with the same operator
        node.op = new_op
    return node


In [ ]:
def bubble_sort(arr):
  """
  Сортирует массив с помощью пузырьковой сортировки.

  Args:
      arr: Список чисел.

  Returns:
      Отсортированный список.
  """
  for i in range(len(arr)):
    for j in range(len(arr) - i - 1):
      if arr[j] > arr[j + 1]:
        arr[j], arr[j + 1] = arr[j + 1], arr[j]
  return arr

def test_bubble_sort():
  assert bubble_sort([5, 2, 4, 1, 3]) == [1, 2, 3, 4, 5]  # Correct test case

survived = mut_test(bubble_sort, test_bubble_sort, size=100)
print(f"Survived Mutants: {survived}")


## 4. Контракты

**4.1.** (уровень сложности: средний)

Изучите работу с модулем deal. Для тестирования контрактов используйте pytest. Выберите одну из программ, содержащих ошибки. Добавьте к программе контракты `pre`, `post`, `ensure`, `raises`, `reason`, `has`.

In [ ]:
import deal
from deal import pre, post, ensure, raises, reason, has
import pytest


def add_numbers(a, b):
    """
    Складывает два числа.

    Args:
        a (int/float): Первое число.
        b (int/float): Второе число.

    Returns:
        int/float: Сумма чисел.
    """
    if not isinstance(a, (int, float)):
        raise TypeError("Аргумент 'a' должен быть числом")
    if not isinstance(b, (int, float)):
        raise TypeError("Аргумент 'b' должен быть числом")
    return a + b


def test_add_numbers_positive_numbers():
    @pre(lambda a, b: isinstance(a, (int, float)) and isinstance(b, (int, float)))
    @post(lambda result: isinstance(result, (int, float)))
    @ensure(lambda a, b, result: result == a + b)
    def test_func(a, b):
        add_numbers(a, b)

    test_func(1, 2)
    test_func(5.5, 3.2)


def test_add_numbers_type_error():
    @raises(TypeError)
    @reason("Аргументы должны быть числами")
    def test_func(a, b):
        add_numbers("abc", 2)

    test_func("abc", 2)


def test_add_numbers_has_return_value():
    @has(lambda result: isinstance(result, (int, float)))
    def test_func(a, b):
        return add_numbers(a, b)

    result = test_func(1, 2)
    assert isinstance(result, (int, float))


**4.2.** (уровень сложности: средний)

Перепишите класс банковского счета (6) с использованием контрактного программирования и, в частности, инвариантов класса. Продемонстрируйте, что реализованные инварианты класса действительно позволяют выявлять ошибки.

In [ ]:
import deal
from deal import invariant

class BankAccount:
    def __init__(self, balance: float):
        if balance < 0:
            raise ValueError("Начальный баланс не может быть отрицательным")
        self._balance = balance

    @invariant
    def has_positive_balance(self) -> bool:
        return self._balance >= 0

    def deposit(self, amount: float):
        if amount <= 0:
            raise ValueError("Сумма депозита должна быть положительной")
        self._balance += amount

    def withdraw(self, amount: float):
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной")
        if amount > self._balance:
            raise ValueError("Недостаточно средств для снятия")
        self._balance -= amount

def test_bank_account():
    account = BankAccount(100)

    # Проверка инварианта
    assert account.has_positive_balance()

    # Депозит
    account.deposit(50)
    assert account.has_positive_balance()

    # Снятие
    account.withdraw(20)
    assert account.has_positive_balance()

    # Попытка снятия больше баланса
    with pytest.raises(ValueError) as e:
        account.withdraw(120)
    assert e.value == "Недостаточно средств для снятия"

    # Попытка депозита отрицательной суммы
    with pytest.raises(ValueError) as e:
        account.deposit(-10)
    assert e.value == "Сумма депозита должна быть положительной"

    # Попытка снятия отрицательной суммы
    with pytest.raises(ValueError) as e:
        account.withdraw(-20)
    assert e.value == "Сумма снятия должна быть положительной"


**4.3.** (уровень сложности: высокий)

Реализуйте контракт, выполнение которого `deal` проверяет статически. Какие ограничения имеют статически проверяемые контракты?

In [ ]:
import deal
from deal import pre

def add_numbers(a: int, b: int) -> int:
    @pre(lambda a, b: isinstance(a, int) and isinstance(b, int))
    def impl(a: int, b: int) -> int:
        return a + b

    return impl(a, b)


## 5. Тестирование на основе свойств

**5.1.** (уровень сложности: средний)

Научитесь работать с библиотекой hypothesis. Протестируйте функцию distance.

In [41]:
!pip install hypothesis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.2/458.2 kB 8.4 MB/s eta 0:00:00


In [45]:
import pytest
from hypothesis import given, strategies as st


def distance(x1, y1, x2, y2):
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

@given(st.floats(), st.floats(), st.floats(), st.floats())
def test_distance_commutative(x1, y1, x2, y2):
    assert distance(x1, y1, x2, y2) == distance(x2, y2, x1, y1)


test_distance_commutative()

ExceptionGroup: Hypothesis found 2 distinct failures. (2 sub-exceptions)

**5.2.** (уровень сложности: средний)

Реализуйте тестирование функций для RLE.

**5.3.** (уровень сложности: высокий)

Реализуйте тестирование для деревьев выражений из предыдущей практики, для одного из «посетителей».


**5.4.** (уровень сложности: высокий)

Используйте тестирование по модели для проверки реализации банковского счета (7).

## 6. Формальная верификация

Формальная верификация головоломок из компьютерных игр

Одной из важных проблем, стоящих перед разработчиком компьютерных игр, является создание интересных головоломок, в которых отсутствуют тупиковые состояния (состояния, из которых нельзя достичь цели).

Формальную верификацию проведем следующим образом:

1. Реализовать игровую ситуацию в виде некоторого количества локаций с указанием перечня возможных действий в каждой из них.
1. Сгенерировать по реализованной игровой ситуации граф всех возможных игровых состояний, в котором ребра задают переходы из состояние в состояние.
1. Проанализировать граф состояний на предмет проверяемого игрового свойства.

Рассмотрим следующую игровую ситуацию из [PuzzleGraph](https://runevision.itch.io/puzzlegraph):

![](data/puzzlegraph.png)

Здесь Goal означает целевое состояние, а состояние с двумя зелеными кругами обозначает старт. Справа от стартового состояния используется специальное ребро, по которому позволяется двигаться только в одну сторону, вправо.

Ниже приведено описание рассмотренной игровой ситуации на Питоне:

In [ ]:
# Функция перехода из комнаты в комнату
def go(room):
    def func(state):
        return dict(state, room=room)
    return func


# Структура игры. Комнаты и допустимые в них действия
game = {
    'room0': dict(
        left=go('room1'),
        up=go('room2'),
        right=go('room3')
    ),
    'room1': dict(
        up=go('room2'),
        right=go('room0')
    ),
    'room2': dict(
    ),
    'room3': dict(
        up=go('room4'),
        right=go('room5')
    ),
    'room4': dict(
        down=go('room3'),
        right=go('room5')
    ),
    'room5': dict(
        up=go('room4'),
        left=go('room3')
    )
}

# Стартовое состояние
START_STATE = dict(room='room0')


def is_goal_state(state):
    '''
    Проверить, является ли состояние целевым.
    '''
    return state['room'] == 'room2'


def get_current_room(state):
    '''
    Выдать комнату, в которой находится игрок.
    '''
    return state['room']

**6.1.** (уровень сложности: средний)

Реализовать функцию `make_model`, которая по структуре игры и стартовому состоянию строит граф всех возможных состояний.

**6.2.** (уровень сложности: высокий)

Реализовать функцию `find_dead_ends`, которая выдает список тупиковых узлов графа. Вспомните тупиковые ситуации из известных вам компьютерных игр, где, иной раз, для дальнейшего прохождения нужно прибегать к старому сохранению или вводу специальных системных команд, и все потому, что был потерян важный для дальнейшего прохождения предмет или не совершено требуемое действие.

**6.3.** (уровень сложности: высокий)

Добавьте в одну из комнат room3-room5 рычаг, нажатие на который делает односторонний переход из room0 в room3 двусторонним.

**6.4.** (уровень сложности: высокий)

Придумайте минимальную игровую ситуацию, при которой будет наблюдаться комбинаторный взрыв для графа состояний.

Простой вариант функции печати графа в формате GraphViz:

In [ ]:
def print_dot(graph, start_key):
    dead_ends = [] # find_dead_ends(graph) TODO
    print('digraph {')
    graph_keys = list(graph.keys())
    for x in graph:
        n = graph_keys.index(x)
        if x == start_key:
            print(f'n{n} [style="filled",fillcolor="dodgerblue",shape="circle"]')
        elif is_goal_state(x):
            print(f'n{n} [style="filled",fillcolor="green",shape="circle"]')
        elif x in dead_ends:
            print(f'n{n} [style="filled",fillcolor="red",shape="circle"]')
        else:
            print(f'n{n} [shape="circle"]')
    for x in graph:
        n1 = graph_keys.index(x)
        for y in graph[x]:
            n2 = graph_keys.index(y)
            print(f'n{n1} -> n{n2}')
    print('}')

Вот как выглядит вывод функции print_dot:
    
![](data/game.svg)

Вывод для пункта 4 с рычагом:
    
![](data/puzzlegraph2.svg)

Есть простая игра The Teeny Tiny Mansion (TTTM), описание которой приведено [здесь](data/tttm.txt). В этой игре имеется 2 персонажа (Алиса и Боб), которыми можно поочередно управлять, 4 комнаты, 3 двери и 3 ключа. Персонажи могут брать ключи и передавать их друг другу. Целью является привести Алису в красную комнату, а Боба — в голубую комнату.

В качестве стартового состояния используйте:

```Python

START_STATE = dict(
    player='alice',
    alice_room='west room',
    bob_room='east room',
    red_key='east room',
    blue_key='west room',
    green_key='east room'
)
```

**6.5.** (уровень сложности: высокий)

Реализовать все пункты предыдущей задачи для TTTM.

**6.6.** (уровень сложности: высокий)

Реализовать функцию подсчета кратчайшего числа шагов, за которые игру можно успешно завершить.

**6.7.** (уровень сложности: хакер)

Провести верификацию для общего случая: Алиса, Боб и 3 ключа случайно располагаются в западной и восточной комнатах.

Пример вывода графа состояний:

![](data/tttm.svg)